### Modelling the problem

We have a set of employees, $E$, and a set of days, $D$.

Let $e \in E$ and $d \in D$.

$\forall e \in E$ and $\forall d \in D$, if employee $e$ has day $d$ off, $L (e,d) = 1$

Therefore $L$ is an $|E|$ by $|D|$ matrix.

$$
L_{e,d} = 
\begin{pmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,|D|} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,|D|} \\
\vdots & \vdots & \ddots & \vdots \\
a_{|E|,1} & a_{|E|,2} & \cdots & a_{|E|,|D|} 
\end{pmatrix}
$$

Let $|E| = 10$ and $|D| = 5$, meaning that there are 10 employees and 5 staff members.

We also have a set of employee quotas $Q$ for each day, which is a matrix of length $ |D| $. 

$Q (d)$ denotes the proportion of staff members who can take annual leave on day $ d $.

Therefore, 
$$\forall d \in D,  \sum_{e=0}^E L(e,d) \leq (|E| * Q(d))$$

Finally, there is a staff leave allowance $A$, which is a matrix of length $|E|$, where $A(e)$ denotes the holiday entitlement remaining for employee $e$.

Therefore, 
$$\forall e \in E, \sum_{d=0}^D L(e,d) \leq A(e)  $$

The preference of leave assignments for all employees in $E$ is given by an $E * D$ matrix $P$, where if $P(e,d) = 1$, the employee has requested this day off.

If an employee does not request a day off, then we do not want the algorithm to assign a day off to them.

Therefore,

$$ L(e,d) \leq P(e,d)  \quad \forall e \in E , \forall d \in D $$



In [ ]:
from ortools.sat.python import cp_model
import numpy as np
import pandas as pd
from math import ceil
import random

In [2]:
num_staff = 10
num_days = 5

staff_limit = [((random.randrange(25, 50, 5) / 100)) for i in range(num_days)] # random capacity for each day
print(f"Staff Limit: {staff_limit}")

staff_leave_allowance = [random.randint(0, 5) for i in range(num_staff)] # random leave allowance for each staff
print(f"Staff Leave Allowance (number of days): {staff_leave_allowance}")

num_ones = int(num_staff * num_days * 0.9)
num_zeros = (num_staff * num_days) - num_ones
array = np.array([1] * num_ones + [0] * num_zeros)
np.random.shuffle(array)
p = array.reshape(num_staff,num_days)
print(f"Preference Matrix: \n {p}")




Staff Limit: [0.45, 0.25, 0.45, 0.35, 0.3]
Staff Leave Allowance (number of days): [1, 2, 2, 0, 3, 4, 1, 5, 1, 3]
Preference Matrix: 
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 0 1]
 [1 0 1 1 1]
 [1 1 1 1 1]
 [1 1 1 0 0]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 0]]


In [3]:
model = cp_model.CpModel()

If employee $e$ has day $d$ off, $L (e,d) = 1$

In [4]:
l = {}
for e in range(num_staff):
    for d in range(num_days):
        l[(e, d)] = model.new_bool_var(f"L_{e}_d{d}")

Ensuring holiday entitlement is not exceeded (denoted by A for each employee).

$\forall e \in E, \sum_{d=0}^D L(e,d) \leq A(e)  $

In [5]:
for e in range(num_staff):
    model.Add(sum(l[e, d] for d in range(num_days)) <= staff_leave_allowance[e])

Ensuring that maximum daily leave quota is not exceeded (denoted by array Q)

$\forall d \in D,  \sum_{e=0}^E L(e,d) \leq (|E| * Q(d))$

In [ ]:

# calculate number of staff * staff_limit for each day
for d in range(num_days):
    model.Add(sum(l[e, d] for e in range(num_staff)) <= ceil(num_staff * staff_limit[d]))

### Multi-Objective solving

Two methods:
- add a weighting for each 
- solve one, then use it as a coefficient and solve again (lexographic solving)

Attempting with method 1

Objective functions:

1) Maximise the leave which matches a persons preference, and minimise leave that doesn't match a preference.

    The line below aims to maximise $P( e , d ) \times L( e , d )$ for each staff member 
    $ e \in E $ and for each day $ d \in d $ .

    If a staff member hasn't asked for a day off ( meaning $ P(e,d) = 0 $ ), the algorithm shouldn't give them a day off, as $ P( e , d ) \times L( e , d ) = 0 \times 1 = 0 $

    Likewise, if a staff member has asked for a day off ( meaning $ P(e,d) = 1 $ ), the algorithm should try to give them a day off, as $ P( e , d ) \times L( e , d ) = 1 \times 1 = 1 $




In [7]:
objective_1 = sum(p[e][d] * l[e, d] for e in range(num_staff) for d in range(num_days))

objective_1_weighting = 1


2) Maximise number of consecutive days and add an upper constraint to the number of consecutive days you can take (soft constraint)

    In order to encourage the algorithm to want to approve "blocks" of annual leave rather than individual days, we aim to maximise the number of consecutive days allocated.

    We can define a variable $S$, where $S(e,d) = L(e,d) * L(e,d+1)$ for $\sum_{d=0}^{|D|-1}$.

    For example, if a person has a preference of two consecutive days $\{1,1\}$, we aim to maximise the product $L(e,d)$ of these two consecutive days.

    Scheduling $\{1,0\}$ or $\{0,1\}$ results in a product of $0$, meaning that this behavour will be discouraged.

In [ ]:
consecutive_days_off = {}

for e in range(num_staff):
    for d in range(num_days-1):
        consecutive_days_off[(e, d)] = model.new_bool_var(f"consecutive_days_off_{e}_{d}")

for e in range(num_staff):
    for d in range(num_days-1):
        model.add_multiplication_equality(consecutive_days_off[(e, d)], [l[e, d], l[e, d+1]])

objective_2 = sum(consecutive_days_off[(e, d)] for e in range(num_staff) for d in range(num_days-1))

objective_2_weighting = 1

3) Prioritise those who have more leave allowance left

In [9]:
objective_3 = sum(staff_leave_allowance[e] * l[e, d] for e in range(num_staff) for d in range(num_days))
objective_3_weighting = 1

4) Prioritise certain staff based on role

5) Prioritise based on children

Join all objectives together with weightings and solve

In [10]:
model.maximize(
                (objective_1_weighting * objective_1) + 
                (objective_2_weighting * objective_2) +
                (objective_3_weighting * objective_3)
            )   

In [ ]:
# Solve model

solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model)

In [12]:

def highlight_cells(val):
    color = '#00ff15' if val == 1 else ''
    return 'color: %s' % color

# function taken from stack overflow
def highlight_diff(data, other, color='#ff616b'):
    attr = f'background-color: {color}'
    return pd.DataFrame(np.where(data.ne(other), attr, ''),
                        index=data.index, columns=data.columns)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

    solutionArray = [[solver.Value(l[s,d]) for d in range(num_days)] for s in range(num_staff)]

    df = pd.DataFrame(solutionArray, columns=[f'Day {d+1}' for d in range(num_days)], index=[f'Employee {x+1}' for x in range(num_staff)])

    df2 = pd.DataFrame(p, columns=[f'Day {d+1}' for d in range(num_days)], index=[f'Employee {x+1}' for x in range(num_staff)])


    print("Solution\nRed cells are cells where preference matrix and solution matrix differ.")
    df_styled = df.style.apply(highlight_diff, axis=None, other=df2).applymap(highlight_cells)
    display(df_styled)

    print("Preference Matrix")
    df2_styled = df2.style.applymap(highlight_cells)
    display(df2_styled)

else:
    print("No feasible solution found.")

Solution
Red cells are cells where preference matrix and solution matrix differ.


C:\Users\seren\AppData\Local\Temp\ipykernel_25596\921478126.py:21: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_styled = df.style.apply(highlight_diff, axis=None, other=df2).applymap(highlight_cells)


,Day 1,Day 2,Day 3,Day 4,Day 5
Employee 1,0,0,0,0,0
Employee 2,0,0,1,1,0
Employee 3,1,0,0,0,1
Employee 4,0,0,0,0,0
Employee 5,0,0,1,1,1
Employee 6,1,1,1,1,0
Employee 7,0,0,0,0,0
Employee 8,1,1,1,1,1
Employee 9,1,0,0,0,0
Employee 10,1,1,1,0,0


Preference Matrix


C:\Users\seren\AppData\Local\Temp\ipykernel_25596\921478126.py:25: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df2_styled = df2.style.applymap(highlight_cells)


,Day 1,Day 2,Day 3,Day 4,Day 5
Employee 1,1,1,1,1,1
Employee 2,1,1,1,1,1
Employee 3,1,1,1,0,1
Employee 4,1,0,1,1,1
Employee 5,1,1,1,1,1
Employee 6,1,1,1,0,0
Employee 7,1,1,1,1,1
Employee 8,1,1,1,1,1
Employee 9,1,1,1,1,1
Employee 10,1,1,1,1,0


In [13]:

class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, num_staff, num_days, l, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._num_staff = num_staff
        self._num_days = num_days
        self._leave = l
        self._solution_count = 0
        self._solution_limit = limit


    def on_solution_callback(self):

        self._solution_count += 1

        print(f"Solution {self._solution_count}")

        for d in range(self._num_days):
            print(f"Day {d+1}")
            for s in range(self._num_staff):
                is_working = False
                if self.value(self._leave[(s, d)]):
                    is_working = True
                    print(f"  Employee {s+1} granted leave")
                if not is_working:
                    print(f"  Employee {s+1} does not work")

        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()



    def solutionCount(self):
        return self._solution_count



In [14]:
solution_limit = 5
solution_printer = SolutionPrinter(num_staff, num_days, l, solution_limit)

solver = cp_model.CpSolver()
# solver.parameters.linearization_level = 0
solver.parameters.enumerate_all_solutions = True

solver.Solve(model, solution_printer)

# Statistics.
print("\nStatistics")
print(f"  - conflicts: {solver.num_conflicts}")
print(f"  - branches : {solver.num_branches}")
print(f"  - wall time: {solver.wall_time}s")

Solution 1
Day 1
  Employee 1 does not work
  Employee 2 granted leave
  Employee 3 granted leave
  Employee 4 does not work
  Employee 5 does not work
  Employee 6 does not work
  Employee 7 granted leave
  Employee 8 granted leave
  Employee 9 granted leave
  Employee 10 does not work
Day 2
  Employee 1 does not work
  Employee 2 does not work
  Employee 3 does not work
  Employee 4 does not work
  Employee 5 granted leave
  Employee 6 granted leave
  Employee 7 does not work
  Employee 8 granted leave
  Employee 9 does not work
  Employee 10 does not work
Day 3
  Employee 1 does not work
  Employee 2 granted leave
  Employee 3 does not work
  Employee 4 does not work
  Employee 5 granted leave
  Employee 6 granted leave
  Employee 7 does not work
  Employee 8 granted leave
  Employee 9 does not work
  Employee 10 granted leave
Day 4
  Employee 1 does not work
  Employee 2 does not work
  Employee 3 does not work
  Employee 4 does not work
  Employee 5 granted leave
  Employee 6 gran